In [1]:
import numpy as np

from HMM import *
from toolbox import *

In [2]:
# Import and separate datasets
ERROR_RATE = 10  # 10% or 20%
train_set, test_set = load_db(error_rate=ERROR_RATE)
X_train = [[token[0] for token in word] for word in train_set]
y_train = [[token[1] for token in word] for word in train_set]
X_test = [[token[0] for token in word] for word in test_set]
y_test = [[token[1] for token in word] for word in test_set]

# Get states and observations sets
states, observations = get_observations_states(X_train, y_train)
print("{} states :\n{}".format(len(states), states))
print("{} observations :\n{}".format(len(observations), observations))

# Example from dataset
print("\nSample example (observation, état) :\n{}".format(train_set[3]))

26 states :
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
26 observations :
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

Sample example (observation, état) :
[('a', 'a'), ('c', 'c'), ('v', 'c'), ('o', 'o'), ('u', 'u'), ('n', 'n'), ('t', 't')]


# hmmlearn

In [3]:
# convert to int and reshape to 1D data
hmm1 = HMM(states, observations, verbose=False)
X_train_id = [hmm1._convert_observations_sequence_to_index(sample) for sample in X_train]
X_train_1D = np.atleast_2d(np.concatenate(X_train_id)).T
X_train_lengths = [len(sample) for sample in X_train_id]

X_test_id = [hmm1._convert_observations_sequence_to_index(sample) for sample in X_test]
y_test_id = [hmm1._convert_observations_sequence_to_index(sample) for sample in y_test]
X_test_1D = np.atleast_2d(np.concatenate(X_test_id)).T
X_test_lengths = [len(sample) for sample in X_test_id]

In [4]:
emission0 = np.eye(26) + 0.2/26
emission0 /= np.sum(emission0, axis=1)

In [4]:
from hmmlearn.hmm import GaussianHMM, MultinomialHMM, GMMHMM

hmm_sk = MultinomialHMM(26, verbose=True, n_iter=5, init_params='ste')
# hmm_sk = MultinomialHMM(26, verbose=True, n_iter=5, init_params='st')
# hmm_sk.emissionprob_ = emission0
hmm_sk.fit(X_train_1D, X_train_lengths)

         1     -466222.6108             +nan
         2     -425584.9953      +40637.6155
         3     -425396.9245        +188.0708
         4     -425143.0750        +253.8495
         5     -424769.2210        +373.8540


MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=26,
        n_iter=5, params='ste',
        random_state=<mtrand.RandomState object at 0x7fb830030948>,
        startprob_prior=1.0, tol=0.01, transmat_prior=1.0, verbose=True)

In [5]:
y_test_pred_sk = [hmm_sk.predict(np.atleast_2d(sample).T) for sample in X_test_id]
y_test_pred = [hmm1._convert_states_sequence_to_string(sample) for sample in y_test_pred_sk]
display_correction_stats(X_test, y_test, y_test_pred, name="HMM_sk")

HMM_sk score on test set
 * accuracy on full words : 0.00%
 * accuracy on letters    : 1.31%
   > typos corrected      : 29 (0.40%)
   > typos not corrected  : 716 (9.78%)
   > typos added          : 6508 (88.91%)

Dummy score on test set
 * accuracy on full words : 62.89%
 * accuracy on letters    : 89.82%
   > typos corrected      : 0 (0.00%)
   > typos not corrected  : 745 (10.18%)
   > typos added          : 0 (0.00%)


In [6]:
np.round(hmm_sk.emissionprob_, 3)

array([[0.002, 0.009, 0.041, 0.004, 0.127, 0.039, 0.011, 0.053, 0.086,
        0.004, 0.015, 0.061, 0.03 , 0.024, 0.085, 0.036, 0.002, 0.089,
        0.062, 0.098, 0.044, 0.015, 0.016, 0.006, 0.036, 0.007],
       [0.052, 0.016, 0.023, 0.026, 0.063, 0.06 , 0.036, 0.053, 0.127,
        0.007, 0.006, 0.006, 0.044, 0.013, 0.074, 0.005, 0.007, 0.073,
        0.075, 0.121, 0.032, 0.011, 0.041, 0.001, 0.026, 0.003],
       [0.051, 0.01 , 0.03 , 0.061, 0.119, 0.028, 0.013, 0.002, 0.073,
        0.   , 0.021, 0.086, 0.013, 0.163, 0.048, 0.02 , 0.004, 0.102,
        0.057, 0.008, 0.041, 0.006, 0.006, 0.009, 0.024, 0.004],
       [0.038, 0.021, 0.082, 0.011, 0.176, 0.059, 0.047, 0.087, 0.047,
        0.008, 0.02 , 0.032, 0.004, 0.027, 0.09 , 0.038, 0.005, 0.054,
        0.08 , 0.038, 0.006, 0.003, 0.01 , 0.006, 0.003, 0.006],
       [0.018, 0.003, 0.05 , 0.009, 0.118, 0.039, 0.013, 0.006, 0.03 ,
        0.005, 0.012, 0.087, 0.029, 0.15 , 0.114, 0.029, 0.003, 0.055,
        0.053, 0.059, 0.031, 0

# Our HMM

In [7]:
# Initialize HMM
hmm1 = HMM(states, observations)

1st order HMM created with: 
 * 26 states
 * 26 observations


In [8]:
# set prior on emission matrix
# emission0 = np.eye(26) + 0.2/26
# emission0 /= np.sum(emission0, axis=1)
# hmm1.observation_logproba = np.log(emission0)

# emission matrix before training
np.round(np.exp(hmm1.observation_logproba), 3)

array([[0.047, 0.028, 0.036, 0.042, 0.033, 0.052, 0.036, 0.049, 0.048,
        0.023, 0.041, 0.054, 0.043, 0.027, 0.026, 0.046, 0.028, 0.045,
        0.037, 0.042, 0.02 , 0.032, 0.049, 0.037, 0.035, 0.042],
       [0.041, 0.042, 0.052, 0.045, 0.052, 0.047, 0.046, 0.029, 0.043,
        0.023, 0.055, 0.025, 0.028, 0.05 , 0.049, 0.02 , 0.038, 0.053,
        0.028, 0.034, 0.03 , 0.028, 0.033, 0.041, 0.042, 0.028],
       [0.039, 0.024, 0.038, 0.036, 0.041, 0.034, 0.024, 0.032, 0.027,
        0.051, 0.029, 0.05 , 0.04 , 0.031, 0.026, 0.024, 0.044, 0.054,
        0.037, 0.027, 0.053, 0.042, 0.053, 0.054, 0.054, 0.037],
       [0.041, 0.034, 0.02 , 0.034, 0.051, 0.038, 0.02 , 0.029, 0.036,
        0.022, 0.047, 0.029, 0.046, 0.03 , 0.045, 0.036, 0.046, 0.053,
        0.048, 0.048, 0.048, 0.02 , 0.048, 0.049, 0.05 , 0.031],
       [0.022, 0.04 , 0.021, 0.048, 0.025, 0.029, 0.041, 0.045, 0.038,
        0.038, 0.052, 0.021, 0.03 , 0.033, 0.046, 0.031, 0.04 , 0.04 ,
        0.05 , 0.026, 0.058, 0

In [9]:
# Train HMM
hmm1.fit(X_train, max_iter=10, tol=0.001)

Unsupervised training by EM algorithm...
EM LOOP: n_iter=1, delta=0.0423
EM LOOP: n_iter=2, delta=0.0690
EM LOOP: n_iter=3, delta=0.0822
EM LOOP: n_iter=4, delta=0.1287
EM LOOP: n_iter=5, delta=0.0727
EM LOOP: n_iter=6, delta=0.0811
EM LOOP: n_iter=7, delta=0.0921
EM LOOP: n_iter=8, delta=0.1071
EM LOOP: n_iter=9, delta=0.0808
EM LOOP: n_iter=10, delta=0.1937


In [10]:
np.round(np.exp(hmm1.observation_logproba), 3)

array([[0.   , 0.   , 0.   , 0.   , 0.   , 0.995, 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.032, 0.038, 0.034, 0.033, 0.032, 0.036, 0.033, 0.032, 0.032,
        0.038, 0.063, 0.032, 0.033, 0.032, 0.032, 0.033, 0.033, 0.032,
        0.032, 0.032, 0.034, 0.035, 0.124, 0.039, 0.033, 0.037],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 1.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.032, 0.038, 0.033, 0.032, 0.032, 0.034, 0.033, 0.032, 0.032,
        0.038, 0.063, 0.032, 0.032, 0.032, 0.032, 0.032, 0.033, 0.032,
        0.032, 0.032, 0.034, 0.034, 0.14 , 0.039, 0.032, 0.036],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0

In [11]:
y_test_pred = hmm1.predict(X_test)
display_correction_stats(X_test, y_test, y_test_pred, name="HMM1")

HMM1 score on test set
 * accuracy on full words : 0.13%
 * accuracy on letters    : 5.55%
   > typos corrected      : 5 (0.07%)
   > typos not corrected  : 740 (10.11%)
   > typos added          : 6174 (84.34%)

Dummy score on test set
 * accuracy on full words : 62.89%
 * accuracy on letters    : 89.82%
   > typos corrected      : 0 (0.00%)
   > typos not corrected  : 745 (10.18%)
   > typos added          : 0 (0.00%)


## Wikipedia example

### our version

In [7]:
states = ['Healthy', 'Fever', 'Z_end']

In [8]:
observations = ['normal', 'cold', 'dizzy']

In [9]:
hmm_test = HMM(states, observations)  
print(hmm_test.omega_Y)
print(hmm_test.omega_X)

1st order HMM created with: 
 * 3 states
 * 3 observations
['Fever', 'Healthy', 'Z_end']
['cold', 'dizzy', 'normal']


In [10]:
hmm_test.transition_logproba = np.array([[0.59, 0.4, 0.01], [0.3, 0.69, 0.01], [0., 0., 1.]])

In [11]:
hmm_test.observation_logproba = np.array([[0.3, 0.6, 0.1], [0.4, 0.1, 0.5], [0.33, 0.33, 0.33]])

In [12]:
hmm_test.initial_state_logproba = np.array([0.4, 0.6, 0.0])

In [13]:
alpha = hmm_test.forward(observations_sequence=['normal', 'cold', 'dizzy'], decode=True)

In [14]:
beta = hmm_test.backward(observations_sequence=['normal', 'cold', 'dizzy'], decode=True)

In [15]:
print(alpha)
print(beta)

[[0.04       0.03408    0.02812032]
 [0.3        0.0892     0.007518  ]
 [0.         0.001122   0.00077708]]
[[0.0035016  0.0384     0.2       ]
 [0.021894   0.0412     0.03333333]
 [0.01249578 0.03707    0.11      ]]


### Wikipedia version

In [16]:
def fwd_bkw(observations, states, start_prob, trans_prob, emm_prob, end_st):
    # forward part of the algorithm
    fwd = []
    f_prev = {}
    for i, observation_i in enumerate(observations):
        f_curr = {}
        for st in states:
            if i == 0:
                # base case for the forward part
                prev_f_sum = start_prob[st]
            else:
                prev_f_sum = sum(f_prev[k]*trans_prob[k][st] for k in states)

            f_curr[st] = emm_prob[st][observation_i] * prev_f_sum

        fwd.append(f_curr)
        f_prev = f_curr

    p_fwd = sum(f_curr[k] * trans_prob[k][end_st] for k in states)
    print(fwd)

    # backward part of the algorithm
    bkw = []
    b_prev = {}
    for i, observation_i_plus in enumerate(reversed(observations[1:]+(None,))):
        b_curr = {}
        for st in states:
            if i == 0:
                # base case for backward part
                b_curr[st] = trans_prob[st][end_st]
            else:
                b_curr[st] = sum(trans_prob[st][l] * emm_prob[l][observation_i_plus] * b_prev[l] for l in states)

        bkw.insert(0,b_curr)
        b_prev = b_curr
    print(bkw)
    p_bkw = sum(start_prob[l] * emm_prob[l][observations[0]] * b_curr[l] for l in states)

    # merging the two parts
    posterior = []
    for i in range(len(observations)):
        posterior.append({st: fwd[i][st] * bkw[i][st] / p_fwd for st in states})

    assert p_fwd == p_bkw
    return fwd, bkw, posterior


In [17]:
states = ('Healthy', 'Fever')
end_state = 'E'
 
observations = ('normal', 'cold', 'dizzy')
 
start_probability = {'Healthy': 0.6, 'Fever': 0.4}
 
transition_probability = {
   'Healthy' : {'Healthy': 0.69, 'Fever': 0.3, 'E': 0.01},
   'Fever' : {'Healthy': 0.4, 'Fever': 0.59, 'E': 0.01},
   }
 
emission_probability = {
   'Healthy' : {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
   'Fever' : {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6},
   }
fwd_bkw(observations,
                   states,
                   start_probability,
                   transition_probability,
                   emission_probability,
                   end_state)

[{'Fever': 0.04000000000000001, 'Healthy': 0.3}, {'Fever': 0.03408, 'Healthy': 0.0892}, {'Fever': 0.028120319999999997, 'Healthy': 0.007518}]
[{'Fever': 0.00109578, 'Healthy': 0.0010418399999999998}, {'Fever': 0.00394, 'Healthy': 0.00249}, {'Fever': 0.01, 'Healthy': 0.01}]


([{'Fever': 0.04000000000000001, 'Healthy': 0.3},
  {'Fever': 0.03408, 'Healthy': 0.0892},
  {'Fever': 0.028120319999999997, 'Healthy': 0.007518}],
 [{'Fever': 0.00109578, 'Healthy': 0.0010418399999999998},
  {'Fever': 0.00394, 'Healthy': 0.00249},
  {'Fever': 0.01, 'Healthy': 0.01}],
 [{'Fever': 0.1229889624426741, 'Healthy': 0.8770110375573259},
  {'Fever': 0.3767719690490461, 'Healthy': 0.623228030950954},
  {'Fever': 0.7890472951586943, 'Healthy': 0.2109527048413057}])

In [18]:
print(alpha)
print(beta)

[[0.04       0.03408    0.02812032]
 [0.3        0.0892     0.007518  ]
 [0.         0.001122   0.00077708]]
[[0.0035016  0.0384     0.2       ]
 [0.021894   0.0412     0.03333333]
 [0.01249578 0.03707    0.11      ]]
